In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession, Window
import pyspark.sql.types as t
import pyspark.sql.functions as f

In [2]:
spark = (SparkSession.builder.master("local").appName("practice2 app").config(conf=SparkConf()).getOrCreate())

In [3]:
spark

In [4]:
# opening business dataset

In [5]:
path = 'yelp_academic_dataset_business.json'

default_business_json = spark.read.json(path)
default_business_json.show()

+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+-------+-------------+--------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|          city|               hours|is_open|     latitude|     longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+-------+-------------+--------------+--------------------+-----------+------------+-----+-----+
|1616 Chapala St, ...|{null, null, null...|Pns2l4eNsfO8kk83d...|Doctors, Traditio...| Santa Barbara|                null|      0|   34.4266787|  -119.7111968|Abby Rappoport, L...|      93101|           7|  5.0|   CA|
|87 Grasso Plaza S...|{null, null, null...|mpf3x-BjTdTEA3yCZ...|Shipping Centers,...|        Affton|{8:0-18:30, 0:0-0...|      1|   

In [6]:
default_business_json.printSchema()

root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

In [7]:
default_business_json.columns

['address',
 'attributes',
 'business_id',
 'categories',
 'city',
 'hours',
 'is_open',
 'latitude',
 'longitude',
 'name',
 'postal_code',
 'review_count',
 'stars',
 'state']

In [76]:
# Кількість рядків та колонок
data = [('rows_num', default_business_json.count()), ('cols_num', len(default_business_json.columns))]
data

[('rows_num', 150346), ('cols_num', 14)]

In [8]:
# transformating default business df

In [9]:
business_df = default_business_json

In [10]:
attributes_cols = [field.name for field in business_df.schema["attributes"].dataType.fields]
for col_name in attributes_cols:
    business_df = business_df.withColumn('attribute_' + col_name, f.col('attributes').getItem(col_name))

In [11]:
hours_cols = [field.name for field in business_df.schema['hours'].dataType.fields]
for col_name in hours_cols:
    business_df = business_df.withColumn('hours_' + col_name, f.col('hours').getItem(col_name))

In [12]:
business_df = business_df.drop('attributes', 'hours')

In [13]:
# changing column names to xx_yy format (from xxYy)
import re
column_names = business_df.columns
new_column_names = [re.sub(r'([a-z])([A-Z])', r'\1_\2', col_name) for col_name in column_names]

# Rename the columns with the new names
for old_name, new_name in zip(column_names, new_column_names):
    business_df = business_df.withColumnRenamed(old_name, new_name.lower())

In [14]:
business_df.show()

+--------------------+--------------------+--------------------+--------------+-------+-------------+--------------+--------------------+-----------+------------+-----+-----+---------------------------+----------------------+-----------------+--------------------+--------------+---------------------+---------------------+----------------------+----------------------------------+---------------------------------------+--------------------------+-----------------------------+----------------+--------------------+-----------------+------------------------------+----------------------+--------------------+--------------------------+-----------------------+-----------------------+-----------------------------+--------------------+----------------+---------------+---------------------+---------------------+-------------------------+----------------------------+-------------------------------------+------------------------------+-------------------------------------+--------------------------

In [15]:
business_df.printSchema()

root
 |-- address: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- city: string (nullable = true)
 |-- is_open: long (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- stars: double (nullable = true)
 |-- state: string (nullable = true)
 |-- attribute_accepts_insurance: string (nullable = true)
 |-- attribute_ages_allowed: string (nullable = true)
 |-- attribute_alcohol: string (nullable = true)
 |-- attribute_ambience: string (nullable = true)
 |-- attribute_byob: string (nullable = true)
 |-- attribute_byobcorkage: string (nullable = true)
 |-- attribute_best_nights: string (nullable = true)
 |-- attribute_bike_parking: string (nullable = true)
 |-- attribute_business_accepts_bitcoin: string (nullable = true)
 |-- attribute_business_accepts_credit

In [16]:
business_df.columns

['address',
 'business_id',
 'categories',
 'city',
 'is_open',
 'latitude',
 'longitude',
 'name',
 'postal_code',
 'review_count',
 'stars',
 'state',
 'attribute_accepts_insurance',
 'attribute_ages_allowed',
 'attribute_alcohol',
 'attribute_ambience',
 'attribute_byob',
 'attribute_byobcorkage',
 'attribute_best_nights',
 'attribute_bike_parking',
 'attribute_business_accepts_bitcoin',
 'attribute_business_accepts_credit_cards',
 'attribute_business_parking',
 'attribute_by_appointment_only',
 'attribute_caters',
 'attribute_coat_check',
 'attribute_corkage',
 'attribute_dietary_restrictions',
 'attribute_dogs_allowed',
 'attribute_drive_thru',
 'attribute_good_for_dancing',
 'attribute_good_for_kids',
 'attribute_good_for_meal',
 'attribute_hair_specializes_in',
 'attribute_happy_hour',
 'attribute_has_tv',
 'attribute_music',
 'attribute_noise_level',
 'attribute_open24hours',
 'attribute_outdoor_seating',
 'attribute_restaurants_attire',
 'attribute_restaurants_counter_service'

In [78]:
# Кількість рядків та колонок
data = [('rows_num', business_df.count()), ('cols_num', len(business_df.columns))]
data

[('rows_num', 150346), ('cols_num', 58)]

In [79]:
path = 'yelp_academic_dataset_review.json'

review_df = spark.read.json(path)

In [80]:
review_df.show()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPpI6HXG530lwP...|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67...|  3.0|Family diner. Had...|     0|8g_iMtfSiwikVnbP2...|
|kxX2SOes4o-D3ZQBk...|   1|2015-01-04 00:01:03|    0|AqPFMleE6RsU23_au...|  5.0|Wow!  Yummy, diff...|     1|_7bHUi9Uuf5__HHc_...|
|e4Vwtrqf-wpJfwesg...|   1|2017-01-14 20:54:15|    0|Sx8TMOWLNuJBWer-0...|  4.0|Cute inter

In [81]:
review_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



In [82]:
review_df.columns

['business_id',
 'cool',
 'date',
 'funny',
 'review_id',
 'stars',
 'text',
 'useful',
 'user_id']

In [83]:
# Кількість рядків та колонок
data = [('rows_num', review_df.count()), ('cols_num', len(review_df.columns))]
data

[('rows_num', 6990280), ('cols_num', 9)]

In [18]:
# Питання №1: Які оцінки мав кожен з бізнесів у кожен рік свого існування?

In [19]:
business_names = business_df.select('business_id', 'name')

In [20]:
review_df = review_df.withColumn('year', f.year(f.to_timestamp('date', 'yyyy-MM-dd HH:mm:ss')))
avg_stars_by_year = (review_df.groupBy('business_id', 'year')
                                .agg(f.avg('stars').alias('average_stars')))

In [21]:
result_df = avg_stars_by_year.join(business_names, 'business_id', 'inner')

In [22]:
result_df.orderBy('name', 'year', ascending=True).show(50)

+--------------------+----+------------------+--------------------+
|         business_id|year|     average_stars|                name|
+--------------------+----+------------------+--------------------+
|nL5Lu2HbjAYClsass...|2018|               3.2|        Grow Academy|
|nL5Lu2HbjAYClsass...|2019|               5.0|        Grow Academy|
|t7vVRlyF9_lxuL1au...|2015|               5.0| Joe's Throwback ...|
|t7vVRlyF9_lxuL1au...|2016|               5.0| Joe's Throwback ...|
|t7vVRlyF9_lxuL1au...|2018|               5.0| Joe's Throwback ...|
|t7vVRlyF9_lxuL1au...|2019|               4.0| Joe's Throwback ...|
|t7vVRlyF9_lxuL1au...|2020|               5.0| Joe's Throwback ...|
|t7vVRlyF9_lxuL1au...|2021|               4.5| Joe's Throwback ...|
|R1nPs3Lc2eXRW95oY...|2013|               4.0| Leland's Barbershop|
|R1nPs3Lc2eXRW95oY...|2014|               5.0| Leland's Barbershop|
|R1nPs3Lc2eXRW95oY...|2015|               5.0| Leland's Barbershop|
|R1nPs3Lc2eXRW95oY...|2019|               5.0| L

In [23]:
# Питання №2: На що потрібно зважати найбільше, при створенні ресторану, підходящого для сімей з дітьми?

In [24]:
business_df.select('categories').show(truncate=False)

+----------------------------------------------------------------------------------------------------------+
|categories                                                                                                |
+----------------------------------------------------------------------------------------------------------+
|Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists|
|Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services                            |
|Department Stores, Shopping, Fashion, Home & Garden, Electronics, Furniture Stores                        |
|Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries                                                     |
|Brewpubs, Breweries, Food                                                                                 |
|Burgers, Fast Food, Sandwiches, Food, Ice Cream & Frozen Yogurt, Restaurants                              |
|Sporting Goods, Fa

In [25]:
good_for_kids_df = business_df.filter(
    (f.col('categories').contains('Restaurants')) &
    (f.col('attribute_good_for_kids') == 'True') &
    (f.col('stars') > 4.0)
)

good_for_kids_df = good_for_kids_df.withColumn("attribute_alcohol",
                                     f.when((f.col("attribute_alcohol").isNull()) |
                                            (f.col("attribute_alcohol") == 'none') |
                                            (f.col("attribute_alcohol") == u'none'),
                                            False).otherwise(True))
good_for_kids_df = good_for_kids_df.withColumn("attribute_smoking",
                                     f.when((f.col("attribute_smoking").isNull()) |
                                            (f.col("attribute_smoking") == 'no') |
                                            (f.col("attribute_smoking") == 'outdoor') |
                                            (f.col("attribute_smoking") == u'no'),
                                            False).otherwise(True))
good_for_kids_df = good_for_kids_df.withColumn("attribute_wi_fi",
                                     f.when((f.col("attribute_wi_fi").isNull()) |
                                            (f.col("attribute_wi_fi") == 'no') |
                                            (f.col("attribute_wi_fi") == u'no'),
                                            False).otherwise(True))

days_of_week = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
for day in days_of_week:
    column_name = f"hours_{day}"
    good_for_kids_df = good_for_kids_df.withColumn(column_name,
                                                   f.when((f.col(column_name).isNull()), False).otherwise(True))

In [26]:
# Загальна кількість записів
total_rows = good_for_kids_df.count()

# Записуємо кількість появ True для кожного з атрибутів
agg_df = good_for_kids_df.agg(*[f.sum(f.when(f.col(col) == 'True', 1).otherwise(0)).alias(f"num_{col}_true") 
             for col in business_df.columns if col.startswith("attribute_") or col.startswith("hours_")])
agg_df = agg_df.withColumn('total_rows', f.lit(total_rows))

In [27]:
for col in business_df.columns:
    if col.startswith('attribute_'):
        col_name = col.replace('attribute_', '')
        agg_df = agg_df.withColumn(f'percent_{col_name}_true', 
                                   f.when(f.col(f'num_{col}_true') != 0, 
                                          f.col(f'num_{col}_true') / 
                                          f.col('total_rows') * 100)
                                   .otherwise(-1))
    if col.startswith('hours_'):
        agg_df = agg_df.withColumn(f'percent_{col}_true', 
                                   f.when(f.col(f'num_{col}_true') != 0, 
                                          f.col(f'num_{col}_true') / 
                                          f.col('total_rows') * 100)
                                   .otherwise(-1))

In [28]:
agg_df = agg_df.drop(*[col for col in agg_df.columns if col.startswith("num_")])
agg_df = agg_df.drop('total_rows')
agg_df.show(vertical=True)

-RECORD 0----------------------------------------------------------
 percent_accepts_insurance_true             | -1.0                 
 percent_ages_allowed_true                  | -1.0                 
 percent_alcohol_true                       | 87.08381171067738    
 percent_ambience_true                      | -1.0                 
 percent_byob_true                          | 3.807883658629927    
 percent_byobcorkage_true                   | -1.0                 
 percent_best_nights_true                   | -1.0                 
 percent_bike_parking_true                  | 64.37045541523153    
 percent_business_accepts_bitcoin_true      | 0.4018369690011481   
 percent_business_accepts_credit_cards_true | 88.27018752391886    
 percent_business_parking_true              | -1.0                 
 percent_by_appointment_only_true           | 0.24875621890547264  
 percent_caters_true                        | 53.36777650210486    
 percent_coat_check_true                    | 0.

In [29]:
# Питання №3: Чи має позитивний вплив byob (дозвіл на власну випивку) на рейтинг закладу?

In [30]:
byob_business_df = business_df.select('stars', 'attribute_byob')
byob_business_df = byob_business_df.dropna(subset=['attribute_byob'])

In [31]:
byob_business_df.show()

+-----+--------------+
|stars|attribute_byob|
+-----+--------------+
|  4.5|         False|
|  4.5|         False|
|  4.0|         False|
|  3.0|          True|
|  4.5|         False|
|  4.0|         False|
|  5.0|         False|
|  4.0|         False|
|  4.5|          True|
|  4.0|         False|
|  4.5|          True|
|  4.0|         False|
|  4.5|          None|
|  4.0|         False|
|  3.0|          True|
|  4.0|         False|
|  4.0|         False|
|  4.0|         False|
|  4.0|         False|
|  4.5|         False|
+-----+--------------+
only showing top 20 rows



In [32]:
byob_true_df = byob_business_df.filter(f.col('attribute_byob') == True)
avg_stars_byob_true = byob_true_df.groupBy('attribute_byob').agg(f.avg('stars').alias('avg_stars_byob'))

In [33]:
byob_false_df = byob_business_df.filter(f.col('attribute_byob') == False)
avg_stars_byob_false = byob_false_df.groupBy('attribute_byob').agg(f.avg('stars').alias('avg_stars_byob'))

In [34]:
avg_stars_byob_true.show()

+--------------+----------------+
|attribute_byob|  avg_stars_byob|
+--------------+----------------+
|          True|4.13872255489022|
+--------------+----------------+



In [35]:
avg_stars_byob_false.show()

+--------------+------------------+
|attribute_byob|    avg_stars_byob|
+--------------+------------------+
|         False|3.9928716904276986|
+--------------+------------------+



In [36]:
result_df = avg_stars_byob_true.union(avg_stars_byob_false)

In [37]:
result_df.show()

+--------------+------------------+
|attribute_byob|    avg_stars_byob|
+--------------+------------------+
|          True|  4.13872255489022|
|         False|3.9928716904276986|
+--------------+------------------+



In [38]:
# Питання №4: Які категорії бізнесів мають найбільше відгуків?

In [39]:
window = Window.partitionBy('categories')

(business_df
 .filter(f.col('categories').isNotNull()).withColumn('reviews', f.sum(f.col('review_count')).over(window))
 .select('categories', 'reviews')
 .distinct()
 .orderBy('reviews', ascending=False)
 .show())

+--------------------+-------+
|          categories|reviews|
+--------------------+-------+
|Mexican, Restaurants|  53043|
|Restaurants, Mexican|  52996|
|  Restaurants, Pizza|  32746|
|  Pizza, Restaurants|  28477|
|Beauty & Spas, Na...|  27484|
|Restaurants, Italian|  26320|
|Italian, Restaurants|  25388|
|Restaurants, Chinese|  23985|
|Nail Salons, Beau...|  23100|
|Chinese, Restaurants|  22071|
|American (New), R...|  18647|
|  Food, Coffee & Tea|  17507|
|  Coffee & Tea, Food|  17194|
|Restaurants, Amer...|  16918|
|   Thai, Restaurants|  16490|
|Seafood, Restaurants|  13356|
|   Restaurants, Thai|  12978|
|Hotels, Event Pla...|  12788|
|Event Planning & ...|  12644|
|Restaurants, Seafood|  10844|
+--------------------+-------+
only showing top 20 rows



In [40]:
# Питання №5: Який вплив має дозвіл на вхід з собаками на оцінку закладу?

In [41]:
window = Window.partitionBy('attribute_dogs_allowed')

dogs_df = business_df.withColumn('attribute_dogs_allowed',
                                     f.when((f.col('attribute_dogs_allowed').isNull()) |
                                            (f.col('attribute_dogs_allowed') == False) |
                                            (f.col('attribute_dogs_allowed') == None),
                                            False).otherwise(True))
(dogs_df.filter(f.col('attribute_dogs_allowed')
                .isNotNull()).withColumn('avg_stars', 
                                         f.sum(f.col('stars')).over(window) / f.count(f.col('stars')).over(window))
 .select('attribute_dogs_allowed', 'avg_stars')
 .distinct()
 .orderBy('avg_stars', ascending=False)
 .show())

+----------------------+-----------------+
|attribute_dogs_allowed|        avg_stars|
+----------------------+-----------------+
|                  true|4.086255609107528|
|                 false|3.576315224244608|
+----------------------+-----------------+



In [42]:
# Питання №6: Який відсоток бізнесів має можливість оплати криптовалютою?

In [67]:
accepts_btc_num = business_df.filter(btc_business_df['attribute_business_accepts_bitcoin'] == True).count()
total_rows_num = business_df.count()

In [73]:
data = [(accepts_btc_num * 100 / total_rows_num,)]
schema = t.StructType([
    t.StructField('accepts_btc_percent', t.FloatType(), False)])
accepts_btc_df = spark.createDataFrame(data, schema)

In [74]:
accepts_btc_df.show()

+-------------------+
|accepts_btc_percent|
+-------------------+
|         0.31261224|
+-------------------+

